## Libraries

In [ ]:
import io
import math
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import re
import time
import numpy as np
import seaborn as sns
import pickle
import datetime
from scipy.stats import linregress
from matplotlib.pyplot import figure
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import NearestNeighbors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Preprocessing

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the data
data = pd.read_csv('/content/drive/MyDrive/TA/NEW DATABASE/THESISMLOTL.csv')

# Define the features and target
features = ['Motor Ampere', 'Frequency', 'Pump Intake Pressure', 'Temperature Motor',
            'Output Volt', 'Pump Discharge Pressure', 'Input Voltage', 'Motor Horse Power (HP)']
target = 'EST RUL'

X = data[features].values
y = data[target].values

# Split the data into training, validation, and test sets
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Reshape data for different models
# CNN-based models
X_train_cnn = X_train_scaled.reshape(-1, 8, 1)
X_val_cnn = X_val_scaled.reshape(-1, 8, 1)
X_test_cnn = X_test_scaled.reshape(-1, 8, 1)

# LSTM-based models
X_train_lstm = X_train_scaled.reshape(-1, 1, 8)
X_val_lstm = X_val_scaled.reshape(-1, 1, 8)
X_test_lstm = X_test_scaled.reshape(-1, 1, 8)

# For combined models (e.g., CNN + LSTM, CNN + Transformer, etc.)
X_train_cnn_lstm = X_train_cnn
X_val_cnn_lstm = X_val_cnn
X_test_cnn_lstm = X_test_cnn

X_train_cnn_cbam = X_train_cnn
X_val_cnn_cbam = X_val_cnn
X_test_cnn_cbam = X_test_cnn

X_train_cnn_trans = X_train_cnn
X_val_cnn_trans = X_val_cnn
X_test_cnn_trans = X_test_cnn

X_train_cnn_cbam_trans = X_train_cnn
X_val_cnn_cbam_trans = X_val_cnn
X_test_cnn_cbam_trans = X_test_cnn

X_train_lstm_trans = X_train_lstm
X_val_lstm_trans = X_val_lstm
X_test_lstm_trans = X_test_lstm

X_train_cnn_lstm_trans = X_train_cnn
X_val_cnn_lstm_trans = X_val_cnn
X_test_cnn_lstm_trans = X_test_cnn

print('Reshaped data for CNN:', X_train_cnn.shape)
print('Reshaped data for LSTM:', X_train_lstm.shape)
print('Reshaped data for CNN + LSTM:', X_train_cnn_lstm.shape)
print('Reshaped data for CNN + CBAM:', X_train_cnn_cbam.shape)
print('Reshaped data for CNN + Transformer:', X_train_cnn_trans.shape)
print('Reshaped data for CNN + CBAM + Transformer:', X_train_cnn_cbam_trans.shape)
print('Reshaped data for LSTM + Transformer:', X_train_lstm_trans.shape)
print('Reshaped data for CNN + LSTM + Transformer:', X_train_cnn_lstm_trans.shape)

Reshaped data for CNN: (25566, 8, 1)
Reshaped data for LSTM: (25566, 1, 8)
Reshaped data for CNN + LSTM: (25566, 8, 1)
Reshaped data for CNN + CBAM: (25566, 8, 1)
Reshaped data for CNN + Transformer: (25566, 8, 1)
Reshaped data for CNN + CBAM + Transformer: (25566, 8, 1)
Reshaped data for LSTM + Transformer: (25566, 1, 8)
Reshaped data for CNN + LSTM + Transformer: (25566, 8, 1)


# MODEL

## CNN

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten

# Define the CNN model
def create_cnn_model():
    model = Sequential()
    model.add(Conv1D(filters=16, kernel_size=2, activation='relu', input_shape=(8, 1), padding='same'))
    model.add(Conv1D(filters=32, kernel_size=2, activation='relu', padding='same'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

cnn_model = create_cnn_model()
cnn_model.summary()

# Train the CNN model
cnn_history = cnn_model.fit(X_train_cnn, y_train, epochs=50, batch_size=32, validation_data=(X_val_cnn, y_val))

# Evaluate the CNN model
cnn_mae, cnn_mse = cnn_model.evaluate(X_test_cnn, y_test)
cnn_rmse = np.sqrt(cnn_mse)
cnn_r2 = 1 - (np.sum((y_test - cnn_model.predict(X_test_cnn).flatten())**2) / np.sum((y_test - np.mean(y_test))**2))

print('CNN Evaluation:')
print(f'MAE: {cnn_mae}, MSE: {cnn_mse}, RMSE: {cnn_rmse}, R2: {cnn_r2}')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 8, 16)             48        
                                                                 
 conv1d_1 (Conv1D)           (None, 8, 32)             1056      
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 17617 (68.82 KB)
Trainable params: 17617 (68.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
799/799 [

## LSTM

In [ ]:
from tensorflow.keras.layers import LSTM

# Define the LSTM model
def create_lstm_model():
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(1, 8)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

lstm_model = create_lstm_model()
lstm_model.summary()

# Train the LSTM model
lstm_history = lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_data=(X_val_lstm, y_val))

# Evaluate the LSTM model
lstm_mae, lstm_mse = lstm_model.evaluate(X_test_lstm, y_test)
lstm_rmse = np.sqrt(lstm_mse)
lstm_r2 = 1 - (np.sum((y_test - lstm_model.predict(X_test_lstm).flatten())**2) / np.sum((y_test - np.mean(y_test))**2))

print('LSTM Evaluation:')
print(f'MAE: {lstm_mae}, MSE: {lstm_mse}, RMSE: {lstm_rmse}, R2: {lstm_r2}')


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                11800     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 11851 (46.29 KB)
Trainable params: 11851 (46.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
799/799 [==============================] - 4s 3ms/step - loss: 846236.3750 - mae: 576.7843 - val_loss: 703657.6250 - val_mae: 507.6960
Epoch 2/50
799/799 [==============================] - 2s 3ms/step - loss: 678508.8125 - mae: 493.8418 - val_loss: 542865.6875 - val_mae: 447.0838
Epoch 3/50
799/799 [==============================] - 3s 3ms/step - loss: 552001.1875 - mae: 467.4388 - val_loss: 476284.1

## CNN + LSTM

In [ ]:
from tensorflow.keras.layers import TimeDistributed

# Define the CNN + LSTM model
def create_cnn_lstm_model():
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=16, kernel_size=2, activation='relu', padding='same'), input_shape=(8, 1, 1)))
    model.add(TimeDistributed(Conv1D(filters=32, kernel_size=2, activation='relu', padding='same')))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

cnn_lstm_model = create_cnn_lstm_model()
cnn_lstm_model.summary()

# Train the CNN + LSTM model
cnn_lstm_history = cnn_lstm_model.fit(X_train_cnn_lstm, y_train, epochs=50, batch_size=32, validation_data=(X_val_cnn_lstm, y_val))

# Evaluate the CNN + LSTM model
cnn_lstm_mae, cnn_lstm_mse = cnn_lstm_model.evaluate(X_test_cnn_lstm, y_test)
cnn_lstm_rmse = np.sqrt(cnn_lstm_mse)
cnn_lstm_r2 = 1 - (np.sum((y_test - cnn_lstm_model.predict(X_test_cnn_lstm).flatten())**2) / np.sum((y_test - np.mean(y_test))**2))

print('CNN + LSTM Evaluation:')
print(f'MAE: {cnn_lstm_mae}, MSE: {cnn_lstm_mse}, RMSE: {cnn_lstm_rmse}, R2: {cnn_lstm_r2}')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 8, 1, 16)          48        
 ributed)                                                        
                                                                 
 time_distributed_1 (TimeDi  (None, 8, 1, 32)          1056      
 stributed)                                                      
                                                                 
 time_distributed_2 (TimeDi  (None, 8, 32)             0         
 stributed)                                                      
                                                                 
 lstm_1 (LSTM)               (None, 50)                16600     
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                      

## CNN + CBAM

In [ ]:
from tensorflow.keras.layers import Add, Activation, GlobalAveragePooling1D, GlobalMaxPooling1D, Reshape, Dense, Multiply, Concatenate

# Define the CBAM block
def cbam_block(input_tensor, ratio=8):
    channel = input_tensor.shape[-1]

    # Channel Attention
    shared_layer_one = Dense(channel//ratio, activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
    shared_layer_two = Dense(channel, kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')

    avg_pool = GlobalAveragePooling1D()(input_tensor)
    avg_pool = Reshape((1, channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    max_pool = GlobalMaxPooling1D()(input_tensor)
    max_pool = Reshape((1, channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)

    cbam_feature = Add()([avg_pool, max_pool])
    cbam_feature = Activation('sigmoid')(cbam_feature)

    cbam_feature = Multiply()([input_tensor, cbam_feature])

    # Spatial Attention
    avg_pool = tf.reduce_mean(cbam_feature, axis=-1, keepdims=True)
    max_pool = tf.reduce_max(cbam_feature, axis=-1, keepdims=True)
    concat = Concatenate(axis=-1)([avg_pool, max_pool])
    cbam_feature = Conv1D(filters=1, kernel_size=7, padding='same', activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(concat)

    cbam_feature = Multiply()([cbam_feature, input_tensor])

    return cbam_feature

# Define the CNN + CBAM model
def create_cnn_cbam_model():
    input_tensor = tf.keras.layers.Input(shape=(8, 1))
    x = Conv1D(filters=16, kernel_size=2, activation='relu', padding='same')(input_tensor)
    x = cbam_block(x)
    x = Conv1D(filters=32, kernel_size=2, activation='relu', padding='same')(x)
    x = cbam_block(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    output_tensor = Dense(1)(x)

    model = tf.keras.Model(inputs=input_tensor, outputs=output_tensor)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

cnn_cbam_model = create_cnn_cbam_model()
cnn_cbam_model.summary()

# Train the CNN + CBAM model
cnn_cbam_history = cnn_cbam_model.fit(X_train_cnn_cbam, y_train, epochs=50, batch_size=32, validation_data=(X_val_cnn_cbam, y_val))

# Evaluate the CNN + CBAM model
cnn_cbam_mae, cnn_cbam_mse = cnn_cbam_model.evaluate(X_test_cnn_cbam, y_test)
cnn_cbam_rmse = np.sqrt(cnn_cbam_mse)
cnn_cbam_r2 = 1 - (np.sum((y_test - cnn_cbam_model.predict(X_test_cnn_cbam).flatten())**2) / np.sum((y_test - np.mean(y_test))**2))

print('CNN + CBAM Evaluation:')
print(f'MAE: {cnn_cbam_mae}, MSE: {cnn_cbam_mse}, RMSE: {cnn_cbam_rmse}, R2: {cnn_cbam_r2}')


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 8, 1)]               0         []                            
                                                                                                  
 conv1d_4 (Conv1D)           (None, 8, 16)                48        ['input_1[0][0]']             
                                                                                                  
 global_average_pooling1d (  (None, 16)                   0         ['conv1d_4[0][0]']            
 GlobalAveragePooling1D)                                                                          
                                                                                                  
 global_max_pooling1d (Glob  (None, 16)                   0         ['conv1d_4[0][0]']        

## CNN + TRANSFORMER

In [ ]:
from tensorflow.keras.layers import MultiHeadAttention
import tensorflow as tf

# Simplified CNN + Transformer model for testing
def create_simple_cnn_trans_model():
    input_tensor = tf.keras.layers.Input(shape=(8, 1))
    x = Conv1D(filters=16, kernel_size=2, activation='relu', padding='same')(input_tensor)
    x = Flatten()(x)
    x = tf.expand_dims(x, axis=1)
    x = MultiHeadAttention(num_heads=2, key_dim=2)(x, x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    output_tensor = Dense(1)(x)

    model = tf.keras.Model(inputs=input_tensor, outputs=output_tensor)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

cnn_trans_model = create_simple_cnn_trans_model()
cnn_trans_model.summary()


# Train the CNN + Transformer model
cnn_trans_history = cnn_trans_model.fit(X_train_cnn_trans, y_train, epochs=50, batch_size=32, validation_data=(X_val_cnn_trans, y_val))

# Evaluate the CNN + Transformer model
cnn_trans_mae, cnn_trans_mse = cnn_trans_model.evaluate(X_test_cnn_trans, y_test)
cnn_trans_rmse = np.sqrt(cnn_trans_mse)
cnn_trans_r2 = 1 - (np.sum((y_test - cnn_trans_model.predict(X_test_cnn_trans).flatten())**2) / np.sum((y_test - np.mean(y_test))**2))

print('CNN + Transformer Evaluation:')
print(f'MAE: {cnn_trans_mae}, MSE: {cnn_trans_mse}, RMSE: {cnn_trans_rmse}, R2: {cnn_trans_r2}')


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 8, 1)]               0         []                            
                                                                                                  
 conv1d_8 (Conv1D)           (None, 8, 16)                48        ['input_2[0][0]']             
                                                                                                  
 flatten_3 (Flatten)         (None, 128)                  0         ['conv1d_8[0][0]']            
                                                                                                  
 tf.expand_dims (TFOpLambda  (None, 1, 128)               0         ['flatten_3[0][0]']           
 )                                                                                          

## CNN + CBAM + TRANSFORMER

In [ ]:
# Define the CNN + CBAM + Transformer model
def create_cnn_cbam_trans_model():
    input_tensor = tf.keras.layers.Input(shape=(8, 1))
    x = Conv1D(filters=16, kernel_size=2, activation='relu', padding='same')(input_tensor)
    x = cbam_block(x)
    x = Conv1D(filters=32, kernel_size=2, activation='relu', padding='same')(x)
    x = cbam_block(x)
    x = Flatten()(x)
    x = tf.expand_dims(x, axis=1)
    x = MultiHeadAttention(num_heads=2, key_dim=2)(x, x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    output_tensor = Dense(1)(x)

    model = tf.keras.Model(inputs=input_tensor, outputs=output_tensor)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

cnn_cbam_trans_model = create_cnn_cbam_trans_model()
cnn_cbam_trans_model.summary()

# Train the CNN + CBAM + Transformer model
cnn_cbam_trans_history = cnn_cbam_trans_model.fit(X_train_cnn_cbam_trans, y_train, epochs=50, batch_size=32, validation_data=(X_val_cnn_cbam_trans, y_val))

# Evaluate the CNN + CBAM + Transformer model
cnn_cbam_trans_mae, cnn_cbam_trans_mse = cnn_cbam_trans_model.evaluate(X_test_cnn_cbam_trans, y_test)
cnn_cbam_trans_rmse = np.sqrt(cnn_cbam_trans_mse)
cnn_cbam_trans_r2 = 1 - (np.sum((y_test - cnn_cbam_trans_model.predict(X_test_cnn_cbam_trans).flatten())**2) / np.sum((y_test - np.mean(y_test))**2))

print('CNN + CBAM + Transformer Evaluation:')
print(f'MAE: {cnn_cbam_trans_mae}, MSE: {cnn_cbam_trans_mse}, RMSE: {cnn_cbam_trans_rmse}, R2: {cnn_cbam_trans_r2}')


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 8, 1)]               0         []                            
                                                                                                  
 conv1d_9 (Conv1D)           (None, 8, 16)                48        ['input_3[0][0]']             
                                                                                                  
 global_average_pooling1d_2  (None, 16)                   0         ['conv1d_9[0][0]']            
  (GlobalAveragePooling1D)                                                                        
                                                                                                  
 global_max_pooling1d_2 (Gl  (None, 16)                   0         ['conv1d_9[0][0]']      

## LSTM + TRANSFORMER

In [ ]:
# Define the LSTM + Transformer model
def create_lstm_trans_model():
    input_tensor = tf.keras.layers.Input(shape=(1, 8))
    x = LSTM(50, activation='relu', return_sequences=True)(input_tensor)
    x = MultiHeadAttention(num_heads=2, key_dim=2)(x, x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    output_tensor = Dense(1)(x)

    model = tf.keras.Model(inputs=input_tensor, outputs=output_tensor)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

lstm_trans_model = create_lstm_trans_model()
lstm_trans_model.summary()

# Train the LSTM + Transformer model
lstm_trans_history = lstm_trans_model.fit(X_train_lstm_trans, y_train, epochs=50, batch_size=32, validation_data=(X_val_lstm_trans, y_val))

# Evaluate the LSTM + Transformer model
lstm_trans_mae, lstm_trans_mse = lstm_trans_model.evaluate(X_test_lstm_trans, y_test)
lstm_trans_rmse = np.sqrt(lstm_trans_mse)
lstm_trans_r2 = 1 - (np.sum((y_test - lstm_trans_model.predict(X_test_lstm_trans).flatten())**2) / np.sum((y_test - np.mean(y_test))**2))

print('LSTM + Transformer Evaluation:')
print(f'MAE: {lstm_trans_mae}, MSE: {lstm_trans_mse}, RMSE: {lstm_trans_rmse}, R2: {lstm_trans_r2}')


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 1, 8)]               0         []                            
                                                                                                  
 lstm_2 (LSTM)               (None, 1, 50)                11800     ['input_4[0][0]']             
                                                                                                  
 multi_head_attention_2 (Mu  (None, 1, 50)                862       ['lstm_2[0][0]',              
 ltiHeadAttention)                                                   'lstm_2[0][0]']              
                                                                                                  
 flatten_7 (Flatten)         (None, 50)                   0         ['multi_head_attention_2

## CNN + LSTM + TRANSFORMER

In [ ]:
# Define the CNN + LSTM + Transformer model
def create_cnn_lstm_trans_model():
    input_tensor = tf.keras.layers.Input(shape=(8, 1))
    x = Conv1D(filters=16, kernel_size=2, activation='relu', padding='same')(input_tensor)
    x = Conv1D(filters=32, kernel_size=2, activation='relu', padding='same')(x)
    x = Flatten()(x)
    x = tf.expand_dims(x, axis=1)
    x = LSTM(50, activation='relu', return_sequences=True)(x)
    x = MultiHeadAttention(num_heads=2, key_dim=2)(x, x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    output_tensor = Dense(1)(x)

    model = tf.keras.Model(inputs=input_tensor, outputs=output_tensor)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

cnn_lstm_trans_model = create_cnn_lstm_trans_model()
cnn_lstm_trans_model.summary()

# Train the CNN + LSTM + Transformer model
cnn_lstm_trans_history = cnn_lstm_trans_model.fit(X_train_cnn_lstm_trans, y_train, epochs=50, batch_size=32, validation_data=(X_val_cnn_lstm_trans, y_val))

# Evaluate the CNN + LSTM + Transformer model
cnn_lstm_trans_mae, cnn_lstm_trans_mse = cnn_lstm_trans_model.evaluate(X_test_cnn_lstm_trans, y_test)
cnn_lstm_trans_rmse = np.sqrt(cnn_lstm_trans_mse)
cnn_lstm_trans_r2 = 1 - (np.sum((y_test - cnn_lstm_trans_model.predict(X_test_cnn_lstm_trans).flatten())**2) / np.sum((y_test - np.mean(y_test))**2))

print('CNN + LSTM + Transformer Evaluation:')
print(f'MAE: {cnn_lstm_trans_mae}, MSE: {cnn_lstm_trans_mse}, RMSE: {cnn_lstm_trans_rmse}, R2: {cnn_lstm_trans_r2}')


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 8, 1)]               0         []                            
                                                                                                  
 conv1d_13 (Conv1D)          (None, 8, 16)                48        ['input_5[0][0]']             
                                                                                                  
 conv1d_14 (Conv1D)          (None, 8, 32)                1056      ['conv1d_13[0][0]']           
                                                                                                  
 flatten_8 (Flatten)         (None, 256)                  0         ['conv1d_14[0][0]']           
                                                                                            

## Evaluation

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluate_model(model, X_test, y_test):
    # Predict the values
    y_pred = model.predict(X_test).flatten()

    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # Print the metrics
    print(f'MAE: {mae}')
    print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')
    print(f'R2: {r2}')

    return mae, mse, rmse, r2


In [ ]:
result = {
    'Model': ['CNN', 'LSTM', 'CNN + LSTM', 'CNN + CBAM', 'CNN + Transformer', 'CNN + CBAM + Transformer', 'LSTM + Transformer', 'CNN + LSTM + Transformer'],
    'MAE': [cnn_mae, lstm_mae, cnn_lstm_mae, cnn_cbam_mae, cnn_trans_mae, cnn_cbam_trans_mae, lstm_trans_mae, cnn_lstm_trans_mae],
    'MSE': [cnn_mse, lstm_mse, cnn_lstm_mse, cnn_cbam_mse, cnn_trans_mse, cnn_cbam_trans_mse, lstm_trans_mse, cnn_lstm_trans_mse],
    'RMSE': [cnn_rmse, lstm_rmse, cnn_lstm_rmse, cnn_cbam_rmse, cnn_trans_rmse, cnn_cbam_trans_rmse, lstm_trans_rmse, cnn_lstm_trans_rmse],
    'R2': [cnn_r2, lstm_r2, cnn_lstm_r2, cnn_cbam_r2, cnn_trans_r2, cnn_cbam_trans_r2, lstm_trans_r2, cnn_lstm_trans_r2]
}

results_df = pd.DataFrame(result)
results_df

,Model,MAE,MSE,RMSE,R2
0,CNN,144999.343750,243.600449,15.607705,0.704924
1,LSTM,257172.953125,339.391052,18.422569,0.476649
2,CNN + LSTM,84681.890625,185.181763,13.608151,0.827671
3,CNN + CBAM,151270.500000,252.114548,15.878115,0.692162
4,CNN + Transformer,176275.062500,299.806244,17.314914,0.641277
5,CNN + CBAM + Transformer,53985.093750,161.277084,12.699491,0.890139
6,LSTM + Transformer,69928.015625,181.304810,13.464947,0.857695
7,CNN + LSTM + Transformer,35076.421875,119.243156,10.919851,0.928619
